In [1]:
# https://github.com/hcw-00/PatchCore_anomaly_detection

%load_ext autoreload
%autoreload 2

from sklearn.random_projection import SparseRandomProjection
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from scipy.ndimage import gaussian_filter
from sklearn.metrics import roc_auc_score
from torch.nn import functional as F
from torchvision import transforms
import pytorch_lightning as pl
from PIL import Image
import numpy as np
import argparse
import shutil
import faiss
import torch
import glob
import cv2
import os, time

from PIL import Image
from sklearn.metrics import roc_auc_score
from torch import nn
import pytorch_lightning as pl
from sklearn.metrics import confusion_matrix
import pickle

from meanshift import *


from sklearn.random_projection import SparseRandomProjection
from sklearn.neighbors import NearestNeighbors
from scipy.ndimage import gaussian_filter



def distance_matrix(x, y=None, p=2):  # pairwise distance of vectors

    y = x if type(y) == type(None) else y

    n = x.size(0)
    m = y.size(0)
    d = x.size(1)

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)

    dist = torch.pow(x - y, p).sum(2)

    return dist


class Id():

    def transform(self, X):
        return X

class NN():

    def __init__(self, X=None, Y=None, p=2):
        self.p = p
        self.train(X, Y)

    def train(self, X, Y):
        self.train_pts = X
        self.train_label = Y

    def __call__(self, x):
        return self.predict(x)

    def predict(self, x):
        if type(self.train_pts) == type(None) or type(self.train_label) == type(None):
            name = self.__class__.__name__
            raise RuntimeError(f"{name} wasn't trained. Need to execute {name}.train() first")

        dist = distance_matrix(x, self.train_pts, self.p) ** (1 / self.p)
        labels = torch.argmin(dist, dim=1)
        return self.train_label[labels]

class KNN(NN):

    def __init__(self, X=None, Y=None, k=3, p=2):
        self.k = k
        super().__init__(X, Y, p)

    def train(self, X, Y):
        super().train(X, Y)
        if type(Y) != type(None):
            self.unique_labels = self.train_label.unique()

    def predict(self, x):


        # dist = distance_matrix(x, self.train_pts, self.p) ** (1 / self.p)
        dist = torch.cdist(x, self.train_pts, self.p)

        knn = dist.topk(self.k, largest=False)


        return knn

def copy_files(src, dst, ignores=[]):
    src_files = os.listdir(src)
    for file_name in src_files:
        ignore_check = [True for i in ignores if i in file_name]
        if ignore_check:
            continue
        full_file_name = os.path.join(src, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, os.path.join(dst,file_name))
        if os.path.isdir(full_file_name):
            os.makedirs(os.path.join(dst, file_name), exist_ok=True)
            copy_files(full_file_name, os.path.join(dst, file_name), ignores)

def prep_dirs(root):
    # make embeddings dir
    # embeddings_path = os.path.join(root, 'embeddings')
    embeddings_path = os.path.join('./', 'embeddings', args.category)
    os.makedirs(embeddings_path, exist_ok=True)
    # make sample dir
    sample_path = os.path.join(root, 'sample')
    os.makedirs(sample_path, exist_ok=True)
    # make source code record dir & copy
    source_code_save_path = os.path.join(root, 'src')
    os.makedirs(source_code_save_path, exist_ok=True)
    copy_files('./', source_code_save_path, ['.git','.vscode','__pycache__','logs','README','samples','LICENSE']) # copy source code
    return embeddings_path, sample_path, source_code_save_path

def embedding_concat(x, y):
    # from https://github.com/xiahaifeng1995/PaDiM-Anomaly-Detection-Localization-master
    B, C1, H1, W1 = x.size()
    _, C2, H2, W2 = y.size()
    s = int(H1 / H2)
    x = F.unfold(x, kernel_size=s, dilation=1, stride=s)
    x = x.view(B, C1, -1, H2, W2)
    z = torch.zeros(B, C1 + C2, x.size(2), H2, W2)
    for i in range(x.size(2)):
        z[:, :, i, :, :] = torch.cat((x[:, :, i, :, :], y), 1)
    z = z.view(B, -1, H2 * W2)
    z = F.fold(z, kernel_size=s, output_size=(H1, W1), stride=s)

    return z

def reshape_embedding(embedding):
    embedding_list = []
    for k in range(embedding.shape[0]):
        for i in range(embedding.shape[2]):
            for j in range(embedding.shape[3]):
                embedding_list.append(embedding[k, :, i, j])
    return embedding_list


def cvt2heatmap(gray):
    heatmap = cv2.applyColorMap(np.uint8(gray), cv2.COLORMAP_JET)
    return heatmap

def heatmap_on_image(heatmap, image):
    if heatmap.shape != image.shape:
        heatmap = cv2.resize(heatmap, (image.shape[0], image.shape[1]))
    out = np.float32(heatmap)/255 + np.float32(image)/255
    out = out / np.max(out)
    return np.uint8(255 * out)

def min_max_norm(image):
    a_min, a_max = image.min(), image.max()
    return (image-a_min)/(a_max - a_min)    


def cal_confusion_matrix(y_true, y_pred_no_thresh, thresh, img_path_list):
    pred_thresh = []
    false_n = []
    false_p = []
    for i in range(len(y_pred_no_thresh)):
        if y_pred_no_thresh[i] > thresh:
            pred_thresh.append(1)
            if y_true[i] == 0:
                false_p.append(img_path_list[i])
        else:
            pred_thresh.append(0)
            if y_true[i] == 1:
                false_n.append(img_path_list[i])

    cm = confusion_matrix(y_true, pred_thresh)
    print(cm)
    print('false positive')
    print(false_p)
    print('false negative')
    print(false_n)
    
#imagenet
mean_train = [0.485, 0.456, 0.406]
std_train = [0.229, 0.224, 0.225]

class STPM(pl.LightningModule):
    def __init__(self, hparams):
        super(STPM, self).__init__()
        
        self.loglist = []

        self.save_hyperparameters(hparams)

        self.init_features()
        def hook_t(module, input, output):
            self.features.append(output)

        self.model = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=True)

        for param in self.model.parameters():
            param.requires_grad = False

        self.model.layer2[-1].register_forward_hook(hook_t)
        self.model.layer3[-1].register_forward_hook(hook_t)

        self.criterion = torch.nn.MSELoss(reduction='sum')

        self.init_results_list()

        self.data_transforms = transforms.Compose([
                        transforms.Resize((args.load_size, args.load_size), Image.ANTIALIAS),
                        transforms.ToTensor(),
                        transforms.CenterCrop(args.input_size),
                        transforms.Normalize(mean=mean_train,
                                            std=std_train)])
        self.gt_transforms = transforms.Compose([
                        transforms.Resize((args.load_size, args.load_size)),
                        transforms.ToTensor(),
                        transforms.CenterCrop(args.input_size)])

        self.inv_normalize = transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255], std=[1/0.229, 1/0.224, 1/0.255])

    def init_results_list(self):
        self.gt_list_px_lvl = []
        self.pred_list_px_lvl = []
        self.gt_list_img_lvl = []
        self.pred_list_img_lvl = []
        self.img_path_list = []        

    def init_features(self):
        self.features = []

    def forward(self, x_t):
        self.init_features()
        _ = self.model(x_t)
        return self.features

    def save_anomaly_map(self, anomaly_map, input_img, gt_img, file_name, x_type):
        if anomaly_map.shape != input_img.shape:
            anomaly_map = cv2.resize(anomaly_map, (input_img.shape[0], input_img.shape[1]))
        anomaly_map_norm = min_max_norm(anomaly_map)
        anomaly_map_norm_hm = cvt2heatmap(anomaly_map_norm*255)

        # anomaly map on image
        heatmap = cvt2heatmap(anomaly_map_norm*255)
        hm_on_img = heatmap_on_image(heatmap, input_img)

        # save images
        cv2.imwrite(os.path.join(self.sample_path, f'{x_type}_{file_name}.jpg'), input_img)
        cv2.imwrite(os.path.join(self.sample_path, f'{x_type}_{file_name}_amap.jpg'), anomaly_map_norm_hm)
        cv2.imwrite(os.path.join(self.sample_path, f'{x_type}_{file_name}_amap_on_img.jpg'), hm_on_img)
        cv2.imwrite(os.path.join(self.sample_path, f'{x_type}_{file_name}_gt.jpg'), gt_img)

    def train_dataloader(self):
        image_datasets = MVTecDataset(root=os.path.join(args.dataset_path,args.category), transform=self.data_transforms, gt_transform=self.gt_transforms, phase='train', n_samples=args.n_samples, fold=args.fold)
        train_loader = DataLoader(image_datasets, batch_size=args.batch_size, shuffle=True, num_workers=0) #, pin_memory=True)
        return train_loader

    def test_dataloader(self):
        test_datasets = MVTecDataset(root=os.path.join(args.dataset_path,args.category), transform=self.data_transforms, gt_transform=self.gt_transforms, phase='test', fold=args.fold)
        test_loader = DataLoader(test_datasets, batch_size=1, shuffle=False, num_workers=0) #, pin_memory=True) # only work on batch_size=1, now.
        return test_loader

    def configure_optimizers(self):
        return None

    def on_train_start(self):
        self.model.eval() # to stop running_var move (maybe not critical)
        self.embedding_dir_path, self.sample_path, self.source_code_save_path = prep_dirs(self.logger.log_dir)
        self.embedding_list = []
    
    def on_test_start(self):
        self.embedding_dir_path, self.sample_path, self.source_code_save_path = prep_dirs(self.logger.log_dir)
        self.index = faiss.read_index(os.path.join(self.embedding_dir_path,'index.faiss'))
        if torch.cuda.is_available():
            res = faiss.StandardGpuResources()
            self.index = faiss.index_cpu_to_gpu(res, 0 ,self.index)
        self.init_results_list()
        
    def training_step(self, batch, batch_idx): # save locally aware patch features
        x, _, _, file_name, _ = batch
        features = self(x)
        embeddings = []
        for feature in features:
            m = torch.nn.AvgPool2d(3, 1, 1)
            embeddings.append(m(feature))
        embedding = embedding_concat(embeddings[0], embeddings[1])
        self.embedding_list.extend(reshape_embedding(np.array(embedding)))

    def training_epoch_end(self, outputs): 
        total_embeddings = np.array(self.embedding_list)
        # Random projection
        self.randomprojector = SparseRandomProjection(n_components='auto', eps=0.9) # 'auto' => Johnson-Lindenstrauss lemma
        self.randomprojector.fit(total_embeddings)
        print("Johnson-Lindenstrauss lemma n_components", self.randomprojector.n_components)

        # Coreset Subsampling
        selector = kCenterGreedy(total_embeddings, 0, 0)
        selected_idx = selector.select_batch(model=self.randomprojector, already_selected=[], N=int(total_embeddings.shape[0]*args.coreset_sampling_ratio))
        # print(selected_idx, int(total_embeddings.shape[0]*args.coreset_sampling_ratio))
        self.embedding_coreset = total_embeddings[selected_idx]
        
        print('initial embedding size : ', total_embeddings.shape)
        print('final embedding size : ', self.embedding_coreset.shape)
        
        # np.save("./embeddings/screw/total_embeddings.npy", total_embeddings) 
        # np.save("./embeddings/screw/core_embeddings.npy", self.embedding_coreset) 
        #faiss
        self.index = faiss.IndexFlatL2(self.embedding_coreset.shape[1])
        self.index.add(self.embedding_coreset) 
        faiss.write_index(self.index,  os.path.join(self.embedding_dir_path,'index.faiss'))


    def test_step(self, batch, batch_idx): # Nearest Neighbour Search
        t0 = time.time()
        x, gt, label, file_name, x_type = batch
        # extract embedding
        features = self(x)
        embeddings = []
        for feature in features:
            m = torch.nn.AvgPool2d(3, 1, 1)
            embeddings.append(m(feature))
        embedding_ = embedding_concat(embeddings[0], embeddings[1])
        embedding_test = np.array(reshape_embedding(np.array(embedding_)))
        score_patches, _ = self.index.search(embedding_test , k=args.n_neighbors)
        anomaly_map = score_patches[:,0].reshape((28,28))
        N_b = score_patches[np.argmax(score_patches[:,0])]
        w = (1 - (np.max(np.exp(N_b))/np.sum(np.exp(N_b))))
        score = w*max(score_patches[:,0]) # Image-level score
        gt_np = gt.cpu().numpy()[0,0].astype(int)
        anomaly_map_resized = cv2.resize(anomaly_map, (args.input_size, args.input_size))
        anomaly_map_resized_blur = gaussian_filter(anomaly_map_resized, sigma=4)
        self.gt_list_px_lvl.extend(gt_np.ravel())
        self.pred_list_px_lvl.extend(anomaly_map_resized_blur.ravel())
        self.gt_list_img_lvl.append(label.cpu().numpy()[0])
        self.pred_list_img_lvl.append(score)
        self.img_path_list.extend(file_name)
        # save images
        x = self.inv_normalize(x)
        input_x = cv2.cvtColor(x.permute(0,2,3,1).cpu().numpy()[0]*255, cv2.COLOR_BGR2RGB)
        self.save_anomaly_map(anomaly_map_resized_blur, input_x, gt_np*255, file_name[0], x_type[0])

    def test_epoch_end(self, outputs):
        print("Total pixel-level auc-roc score :")
        pixel_auc = roc_auc_score(self.gt_list_px_lvl, self.pred_list_px_lvl)
        print(pixel_auc)
        print("Total image-level auc-roc score :")
        img_auc = roc_auc_score(self.gt_list_img_lvl, self.pred_list_img_lvl)
        print(img_auc)
        print('test_epoch_end')
        values = {'pixel_auc': pixel_auc, 'img_auc': img_auc}
        self.log_dict(values)
        self.loglist.append(values)
        # anomaly_list = []
        # normal_list = []
        # for i in range(len(self.gt_list_img_lvl)):
        #     if self.gt_list_img_lvl[i] == 1:
        #         anomaly_list.append(self.pred_list_img_lvl[i])
        #     else:
        #         normal_list.append(self.pred_list_img_lvl[i])

        # # thresholding
        # # cal_confusion_matrix(self.gt_list_img_lvl, self.pred_list_img_lvl, img_path_list = self.img_path_list, thresh = 0.00097)
        # # print()
        # with open(args.project_root_path + r'/results.txt', 'a') as f:
        #     f.write(args.category + ' : ' + str(values) + '\n')

def get_args():
    parser = argparse.ArgumentParser(description='ANOMALYDETECTION')
    parser.add_argument('--phase', choices=['train','test'], default='train')
    parser.add_argument('--dataset_path', default=r'../../../../nas-files/mvtec/') # 'D:\Dataset\mvtec_anomaly_detection')#
    #parser.add_argument('--dataset_path', default="/home/ios/data/mvtec")
    parser.add_argument('--category', default='screw')
    parser.add_argument('--num_epochs', default=1)
    parser.add_argument('--batch_size', default=32)
    parser.add_argument('--load_size', default=256) 
    parser.add_argument('--input_size', default=224)
    parser.add_argument('--coreset_sampling_ratio', default=0.1)
    parser.add_argument('--project_root_path', default=r'./test') # 'D:\Project_Train_Results\mvtec_anomaly_detection\210624\test') #
    parser.add_argument('--save_src_code', default=True)
    parser.add_argument('--save_anomaly_map', default=True)
    parser.add_argument('--n_samples', type=int, default=10000)
    parser.add_argument('--n_neighbors', type=int, default=9)
    parser.add_argument('--fold', type=int, default=-1)
    args = parser.parse_args("")
    return args

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

args = get_args()

/home/matthias/Desktop/workspace/mean-shift/meanshift/MeanShift.py:9: SyntaxWarning: "is" with a literal. Did you mean "=="?
  net, layer_map = get_fmap(features) if type(features) is "str" else features


In [2]:
clazzes = ["bottle","carpet","leather","screw","transistor","cable","grid","pill","tile","wood","capsule","hazelnut","metal_nut","toothbrush","zipper"]
mapping = [0,1,2,13,14,6,7,3,4,5,10,11,12,8,9]

# Timing

In [3]:
trainer = pl.Trainer.from_argparse_args(args, default_root_dir=os.path.join(args.project_root_path, args.category), max_epochs=args.num_epochs, gpus=1) #, check_val_every_n_epoch=args.val_freq,  num_sanity_val_steps=0) # ,fast_dev_run=True)
model = STPM(hparams=args)
model = model.to("cuda")
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /home/matthias/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /home/matthias/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth


  0%|          | 0.00/132M [00:00<?, ?B/s]

/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/pytorch_lightning/trainer/optimizers.py:37: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(

  | Name          | Type      | Params
--------------------------------------------
0 | model         | ResNet    | 68.9 M
1 | criterion     | MSELoss   | 0     
2 | inv_normalize | Normalize | 0     
--------------------------------------------
0         Trainable params
68.9 M    Non-trainable params
68.9 M    Total params
275.533   Total estimated model params size (MB)
/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.

Training: 0it [00:00, ?it/s]

/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:145: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn(


Johnson-Lindenstrauss lemma n_components auto
Getting transformed features...
Calculating distances... (250880, 306)


/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [19]:
model = model.to("cuda")
next(model.parameters()).device

device(type='cuda', index=0)

In [20]:
for batch in model.test_dataloader():

    x, gt, label, file_name, x_type = batch
    # extract embedding
    features = model(x.to(device))
    embeddings = []
    for feature in features:
        m = torch.nn.AvgPool2d(3, 1, 1)
        embeddings.append(m(feature))
    embedding_ = embedding_concat(embeddings[0], embeddings[1])
    embedding_test = np.array(reshape_embedding(np.array(embedding_)))
    
    t0 = time.time()
    score_patches, _ = model.index.search(embedding_test , k=args.n_neighbors)
    anomaly_map = score_patches[:,0].reshape((28,28))
    N_b = score_patches[np.argmax(score_patches[:,0])]
    w = (1 - (np.max(np.exp(N_b))/np.sum(np.exp(N_b))))
    score = w*max(score_patches[:,0]) # Image-level score
    fps = 1/(time.time() - t0)        
    print("FPS", fps)
    break

FPS 6.360894707069726


In [23]:
shutil.rmtree("./test")
shutil.rmtree("./embeddings")

# Data-Efficiency

In [9]:

args = get_args()

log = []

for runs in range(3):
    for n in [1, 5, 10, 25, 50, 80, 100]:

        for clazz in ["bottle","carpet","leather","screw","transistor","cable","grid","pill","tile","wood","capsule","hazelnut","metal_nut","toothbrush","zipper"]:

            args.category = clazz
            args.n_samples = n
            args.fold = runs
            args.coreset_sampling_ratio = 0.1

            trainer = pl.Trainer.from_argparse_args(args, default_root_dir=os.path.join(args.project_root_path, args.category), max_epochs=args.num_epochs, gpus=1) #, check_val_every_n_epoch=args.val_freq,  num_sanity_val_steps=0) # ,fast_dev_run=True)
            model = STPM(hparams=args)
            trainer.fit(model)
            trainer.test(model)

            log.append(model.loglist[0])
            #print(log)

            shutil.rmtree("./test")
            shutil.rmtree("./embeddings")

            import gc
            del model
            gc.collect()
            torch.cuda.empty_cache()

        print( np.asarray([l["img_auc"] for l in log]).reshape(-1, 15).mean(1) )

In [18]:
np.asarray([l["img_auc"] for l in log]).reshape(3, 7, 15).mean((0,2)), np.asarray([l["img_auc"] for l in log]).reshape(3, 7, 15).mean(2).std(0)

(array([0.82072766, 0.89863189, 0.9241965 , 0.95029004, 0.96676529,
        0.9789549 , 0.98068434]),
 array([0.00517104, 0.0096512 , 0.00248153, 0.0019344 , 0.00240823,
        0.00124494, 0.0018492 ]))

# Performance

In [30]:
args = get_args()

log = []

for runs in range(5):

    for clazz in ["bottle","carpet","leather","screw","transistor","cable","grid","pill","tile","wood","capsule","hazelnut","metal_nut","toothbrush","zipper"]:

        args.category = clazz
        args.n_samples = 500000
        args.fold = runs

        trainer = pl.Trainer.from_argparse_args(args, default_root_dir=os.path.join(args.project_root_path, args.category), max_epochs=args.num_epochs, gpus=1) #, check_val_every_n_epoch=args.val_freq,  num_sanity_val_steps=0) # ,fast_dev_run=True)
        model = STPM(hparams=args)
        trainer.fit(model)
        trainer.test(model)

        log.append(model.loglist[0])
        #print(log)

        shutil.rmtree("./test")
        shutil.rmtree("./embeddings")
        
        import gc
        del model
        gc.collect()
        torch.cuda.empty_cache()
        
    print( np.asarray([l["img_auc"] for l in log]).reshape(-1, 15).mean(0).mean(), np.asarray([l["img_auc"] for l in log]).reshape(-1, 15).mean(1).std() )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Using cache found in /home/matthias/.cache/torch/hub/pytorch_vision_v0.9.0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type      | Params
--------------------------------------------
0 | model         | ResNet    | 68.9 M
1 | criterion     | MSELoss   | 0     
2 | inv_normalize | Normalize | 0     
--------------------------------------------
0         Trainable params
68.9 M    Non-trainable params
68.9 M    Total params
275.533   Total estimated model params size (MB)


../../../../nas-files/mvtec/bottle/train
['../../../../nas-files/mvtec/bottle/test/good/005.png', '../../../../nas-files/mvtec/bottle/test/good/006.png', '../../../../nas-files/mvtec/bottle/test/good/012.png', '../../../../nas-files/mvtec/bottle/test/good/003.png', '../../../../nas-files/mvtec/bottle/test/good/019.png', '../../../../nas-files/mvtec/bottle/test/good/018.png', '../../../../nas-files/mvtec/bottle/test/good/016.png', '../../../../nas-files/mvtec/bottle/test/good/015.png', '../../../../nas-files/mvtec/bottle/test/good/007.png', '../../../../nas-files/mvtec/bottle/test/good/013.png', '../../../../nas-files/mvtec/bottle/test/good/004.png', '../../../../nas-files/mvtec/bottle/test/good/001.png', '../../../../nas-files/mvtec/bottle/test/good/000.png', '../../../../nas-files/mvtec/bottle/test/good/011.png', '../../../../nas-files/mvtec/bottle/test/good/014.png', '../../../../nas-files/mvtec/bottle/test/good/010.png', '../../../../nas-files/mvtec/bottle/test/good/002.png', '../..

/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:428: UserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:145: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn(


...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


../../../../nas-files/mvtec/zipper/test


/home/matthias/.conda/envs/pytorch3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

Total pixel-level auc-roc score :
0.9835991839438986
Total image-level auc-roc score :
0.9992121848739496
test_epoch_end
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'img_auc': 0.9992122054100037, 'pixel_auc': 0.9835991859436035}
--------------------------------------------------------------------------------
0.9852704271713676 0.0019670742352748804


In [48]:
print( np.asarray([l["img_auc"] for l in log]).reshape(-1, 15).mean(0).mean(), np.asarray([l["img_auc"] for l in log]).reshape(-1, 15).mean(1).std() )
for i in range(15):
    m = np.asarray([l["img_auc"] for l in log]).reshape(-1, 15).mean(0)[mapping][i]
    s = np.asarray([l["img_auc"] for l in log]).reshape(-1, 15).std(0)[mapping][i]
    print(f"{m:.3f}", f"±{s:.3f}") 

0.9852704271713676 0.0019670742352748804
1.000 ±0.000
0.998 ±0.004
1.000 ±0.000
0.983 ±0.010
0.998 ±0.001
0.982 ±0.006
0.961 ±0.008
0.909 ±0.013
0.997 ±0.002
0.995 ±0.003
0.980 ±0.009
1.000 ±0.000
0.994 ±0.002
0.987 ±0.003
0.997 ±0.003
